<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Python-Notebook-Banners/Examples.png"  style="display: block; margin-left: auto; margin-right: auto;";/>
</div>

# Examples: Improving a logistic regression model
© ExploreAI Academy

In this train, we'll rebuild and evaluate logistic regression models through performance metrics to enhance prediction accuracy.

## Learning objectives

By the end of this train, you should be able to:
- Rebuild a logistic regression model for improved accuracy.
- Assess model performance accurately using the confusion matrix and classification report.
- Understand the evaluation of binary classifiers, particularly in the context of class imbalance, and become familiar with metrics such as accuracy, precision, recall, and F1 score.

## Examples

This train is structured as follows:

1. Rebuild our Wisconsin Breast Cancer classifier.
2. Use feature subset selection to create a sparse model.
3. Tune the logistic regression model hyperparameter.

### Example 1: Rebuilding the Breast Cancer classifier

Let's explore the potential of logistic regression in diagnosing breast cancer, utilising the **Wisconsin Breast Cancer dataset**. This dataset includes 30 continuous features and a binary target indicating tumour malignancy. Our aim is to enhance model precision in distinguishing between benign (1) and malignant (0) cases.

We'll start by making the necessary imports:

* `numpy` for numerical operations.
* `pandas` for data manipulation.
* `matplotlib` for plotting graphs.  
* `%matplotlib inline` to ensure that plots are displayed inline within the notebook. 
* `load_breast_cancer` function provides access to the Wisconsin Breast Cancer dataset.
* `classification_report` function for assessing the performance of classification models by providing key metrics such as precision, recall, F1 score, and accuracy for each class. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Import dataset
from sklearn.datasets import load_breast_cancer

# For precision, recall, etc.
from sklearn.metrics import classification_report

We'll then load the breast cancer dataset by calling the `load_breast_cancer` function with the parameter `return_X_y=True`. This instructs the function to return the dataset split into features and target variables, which are assigned to `X` and `y`, respectively. 

In [2]:
X, y = load_breast_cancer(return_X_y=True)

Remember to perform data scaling of the data during preprocessing for all machine learning tasks as _scaling_ puts all our features into the same range.

In [3]:
# Import the scaler module
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()

# Scale data
X_scaled = scaler.fit_transform(X)

We can now split the scaled features `X_scaled` and target `y` into training and testing sets. We'll reserve 20% of the data for testing (`test_size=0.2`) and ensure reproducibility by setting a `random_state`. 

In [4]:
from sklearn.model_selection import train_test_split

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=10)

In [5]:
from sklearn.linear_model import LogisticRegression

### Example 2: Using feature subset selection to create a sparse model

In general, given a particular model and a dataset, there are two categories of change we can make to try and improve the model once it has been trained with default parameters:

1. We can change/reduce which features the model was trained on.
2. We can tweak what are known as model hyperparameters.

For both methods, the benchmark performance metrics we will use (within classification tasks) are precision, recall, and F1 score. In other words, after making our changes, we will compare the results for each of those metrics to the default model to determine if an improvement has been made.

We'll start with the first method and change which features the model is trained on. Assuming the default model was trained using all the features, any change we undertake will invariably reduce the number of features we use. A model trained using some subset of all the available features is known as a _sparse model_.

We'll create four models to compare. In so doing, we'll cover some of the feature subset selection methods available for classification that you may be able to use in the course and your career going forward:

1. Full model – using all features (reference model).
2. Variance thresholding.
3. Select k-best features.
4. Sequential forward selection.

An important note here: Always split your dataset into train/test folds _before_ performing feature selection. This prevents what is known as data leakage, where some of the 'unseen' data are exposed to the model during the training process, which would in turn inflate its test set accuracy later on. Take a quick read through the [tweet below](https://twitter.com/jmschreiber91/status/1137464236156702720?s=20) for some more context.

<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/split-tweet.jpg" alt="train/test split ordering tweet" width="400px;">

For each method, we'll perform the feature selection on both the train and test sets – ensuring the model is tested on data with the same features it was trained on.

#### 1. Full model (no selection)

In order to have a reference model, we'll build a logistic regression classifier using all the features.

In [6]:
lm_full = LogisticRegression()

We'll need to standardise our features here to ensure the model is able to converge.

In [7]:
# No transformations necessary
lm_full.fit(X_train, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [8]:
# Generate predictions from full model
pred_lm_full = lm_full.predict(X_test)

#### 2. Selection by variance thresholding

As you may recall, variance thresholding is a process by which we remove features which don't meet some minimum level of variance.

<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/sketch-var-thresh.png" alt="sketch-var-thresh" width="500px;"/>

In [9]:
# lm instance for var thresh
lm_vt = LogisticRegression()

When we apply the transformation (i.e. run feature selection) on the training set, we are fitting the transformation: `fit_transform(train_data)`. In order to apply the same transformation to the test set, we only need to transform it, not fit it: `transform(test_data)`.

In [10]:
# Import the var thresh model and choose a threshold
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.02)

# Transform (i.e.: run selection on) the training data
X_train_vt = selector.fit_transform(X_train)

In [11]:
X_train_vt.shape

(455, 15)

Using a variance threshold of 0.02, we have managed to trim the number of features in the training dataset from 30 down to 15.

In [12]:
# Fit model to the transformed data
lm_vt.fit(X_train_vt, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [13]:
# Now we'll need to remove the same features we did in the thresholding process
X_test_vt = selector.transform(X_test)

# Generate predictions from var thresh model
pred_lm_vt = lm_vt.predict(X_test_vt)

#### 3. Selection by choosing k-best features

`SelectKBest` is a feature selection module in `sklearn` which removes all but the _k_ highest scoring features according to some specified statistical test. In this case, we'll use `f_classif` as the test, which we specify in the `score_func` argument in the `SelectKBest` method.

`f_classif` computes the ANOVA f-value between each label and feature – read up [more on it here](https://www.statisticshowto.com/probability-and-statistics/f-statistic-value-test/).

<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/sketch-kbest.png" alt="sketch-kbest" width="600px;"/>

In [14]:
# lm instance for k-best
lm_kbest = LogisticRegression()

In [15]:
# Import the feature selector module
from sklearn import feature_selection
from sklearn.feature_selection import f_classif

# Set up selector, choosing score function and number of features to retain
selector_kbest = feature_selection.SelectKBest(score_func=f_classif, k=20)

# Transform (i.e.: run selection on) the training data
X_train_kbest = selector_kbest.fit_transform(X_train, y_train)

In [16]:
X_train_kbest.shape

(455, 20)

In [17]:
# Fit model to the transformed data
lm_kbest.fit(X_train_kbest, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [18]:
# Before generating predictions, we'll need to transform the test data the same way we did the train data
X_test_kbest = selector_kbest.transform(X_test)

# Generate predictions from var thresh model
pred_lm_kbest = lm_kbest.predict(X_test_kbest)

#### 4. Selection using sequential forward selection

In this process, we iteratively fit the logistic regression classifier to our dataset, where our dataset starts with one feature and adds one feature per iteration. For each iteration, k-fold cross-validation is used to compute the performance of the classifier. Thereafter, we can determine which number of features resulted in the optimal performance.

| 🚩 **Note** 🚩 |
| : ----------------------- :|
| We'll be using the `SequentialFeatureSelector` from a library known as `mlxtend`. **Be sure to: "`pip install mlxtend`" before you run the code below**. We'll also be using F1 score to compute the performance of each fitted model. |



<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/sketch-sfs.png" alt="sketch-sfs" width="700px;">

In [19]:
# We begin by declaring the classifier we wish to use
lm_sfs = LogisticRegression()

In [20]:
# Import the selector module and the accuracy_score module to computer performance
from sklearn.metrics import f1_score
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

Take note of the arguments we pass to the SequentialFeatureSelector instance:

- `lm_sfs` is the classifier we want it to use to fit to the data.
- `k-features` is the maximum number of features we want to end up with. We set it higher rather than lower – in case the optimal number of features is higher than we may have thought.
- `forward=True` sets the selector to use forward, as opposed to backward, selection.
- `scoring='f1'` ensures performance is measured using an appropriate metric: here, F1 score.
- `cv=10` means validation will be performed using k-fold cross-validation, with k=10.

In [21]:
# We then build our forward feature selector
sfs = sfs(lm_sfs, k_features=25, forward=True, scoring='f1', cv=10)

The code cell below takes about a minute to execute.

In [ ]:
sfs = sfs.fit(X_train, y_train)

In [22]:
# Plot the results
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

fig1 = plot_sfs(sfs.get_metric_dict(), kind='std_dev')

plt.ylim([0.9, 1])
plt.title('Sequential Forward Selection (w. StdDev)')
plt.grid()
plt.show()

AttributeError: SequentialFeatureSelector has not been fitted, yet.

Plotting the results we can see that performance begins to drop off after the 21st feature is added. Let's lock in those 21 features and use them to train our classifier. To be clear, these are not necessarily the first 21 features, but the 21 features which, when combined, produce optimal performance.

In [ ]:
# Extract the feature names for the optimal number using the dictionary provided by the subsets_ method
columns = list(sfs.subsets_[21]['feature_idx'])

Having determined which number of features was best, we'll select those specific features and train the model using only them. Remember, we need to select the features for both train _and_ test sets.

In [ ]:
X_train_sfs = X_train[:, columns]
X_test_sfs = X_test[:, columns]

In [ ]:
X_train_sfs.shape

Now we can train the model on this reduced training set.

In [ ]:
lm_sfs.fit(X_train_sfs, y_train)

Finally, we'll generate some predictions.

In [ ]:
pred_lm_sfs = lm_sfs.predict(X_test_sfs)

### Assessment

Using `classification report`, we'll look at F1 score, precision, and recall for each of our models.

In [ ]:
print('Full Model')
print(classification_report(y_test, pred_lm_full, target_names=['Malignant', 'Benign']))

print('Var Thresh Model')
print(classification_report(y_test, pred_lm_vt, target_names=['Malignant', 'Benign']))

print('KBest Model')
print(classification_report(y_test, pred_lm_kbest, target_names=['Malignant', 'Benign']))

print('SFS Model')
print(classification_report(y_test, pred_lm_sfs, target_names=['Malignant', 'Benign']))

In [ ]:
print('SFS Model')
print(classification_report(y_test, pred_lm_sfs, target_names=['Malignant', 'Benign']))

Interestingly, the full model achieves perfect accuracy in both classes, a 0.07% improvement over the full model in the previous train. The only difference here is that we scaled our data first – which clearly made a difference for the convergence of the logistic regression classifier.

The other three models all perform very well, with the `SelectKBest` model also achieving perfect accuracy in both classes. 

The variance thresholding model had the poorest performance of the four. Perhaps using variance alone was not a sufficient indicator of the importance of the features in question.

There is yet another method we can use to try and improve the model – let's take a look.

## 3. Tuning the model hyperparameter

Most models have settings or configuration values, known as hyperparameters, that can be changed to modify the fit of the model on the training data. The logistic regression model contains a hyperparameter _C_, which is used to control the penalty we apply to features that are less important (i.e. more important features will have greater weight).

The smaller the value of _C_, the greater the penalty to less important features. _C_ is a value greater than 0. You will recall from regression the regularisation methods Ridge and LASSO, which sought to minimise the L2- or L1-norm, respectively, by shrinking the coefficients of certain features. The penalty term had a coefficient of its own, which we called $\alpha$, which controlled the severity of shrinkage that occurred. The _C_ value we will change here is precisely the _inverse_ value of $\alpha$. In other words, the smaller the value of C, the greater the shrinkage that occurs.

### Training

We create three instances of the `LogisticRegression()` object, specifying a different value of the hyperparameter _C_ for each. When we set C to be small, we'll also specify the penalty type to be `l1` – this will allow some of the coefficients to be shrunk all the way to zero.

In [ ]:
# Small C
model_1 = LogisticRegression(C=0.05, penalty='l1', solver='liblinear', verbose=1)

# Default C
model_2 = LogisticRegression(C=1)

# Large C
model_3 = LogisticRegression(C=10)

In [ ]:
model_1.fit(X_train, y_train)

In [ ]:
model_2.fit(X_train, y_train)

In [ ]:
model_3.fit(X_train, y_train)

Taking a look at the magnitudes of the coefficients of each trained model, we can see that the smaller C is, the more features are shrunk. In the case where `penalty='l1'`, smaller C values literally mean fewer features are included. Notice how only one non-zero coefficient remains in that case.

In [ ]:
model_1.coef_

In [ ]:
model_2.coef_

In [ ]:
model_3.coef_

### Predicting

The models have been trained. Let's make some predictions.

In [ ]:
pred_1 = model_1.predict(X_test)
pred_2 = model_2.predict(X_test)
pred_3 = model_3.predict(X_test)

### Testing

We will once again use the classification report to compare the model results. 

In [ ]:
print('C = 0.05')
print(classification_report(y_test, pred_1, target_names=['Malignant', 'Benign']))

print('C = 1')
print(classification_report(y_test, pred_2, target_names=['Malignant', 'Benign']))

print('C = 10')
print(classification_report(y_test, pred_3, target_names=['Malignant', 'Benign']))

Clearly, in setting C = 0.05 we have tuned the hyperparameter too aggressively and adversely affected the model's performance. Note how the F1 score is significantly lower for that variation than any other we have looked at.

The logistic regression algorithm is a simple case for hyperparameter tuning because it only contains one value to tweak. Other models, like support vector machines, tree-based methods, and neural networks, contain many more hyperparameters – in most cases too many for the model architect to optimise on their own. In these cases, we use what is known as grid search: an iterative testing of multiple combinations of hyperparameters to find the optimal set. More on that later on.

## Conclusion

In this train, we have seen or been introduced to:

- The importance of carrying out a train/test split before doing any feature selection.
- Feature subset selection by variance thresholding.
- Feature subset selection by selecting the k-best features.
- Feature subset selection by sequential forward selection.
- Improving a model by tuning the hyperparameters.

#  

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>